In [ ]:
#feature worth OHE

def ohe(X_train,X_val, ohe_feat):
    OHE = OneHotEncoder(sparse=False)
    if len(ohe_feat) ==1:
        OHE.fit(pd.concat([X_train[ohe_feat], X_val[ohe_feat]], axis=0).values.reshape(-1,1))
        return OHE.transform(X_train[ohe_feat].values.reshape(-1,1)), OHE.transform(X_val[ohe_feat].values.reshape(-1,1))
    else:
        OHE.fit(pd.concat([X_train[ohe_feat], X_val[ohe_feat]], axis=0))
        return OHE.transform(X_train[ohe_feat]), OHE.transform(X_val[ohe_feat])
    
    
def fw_feature_selection(X_train,y_train, X_val, y_val, loss=True): #need to drop last feat
    score_hist = []
    selected_feature = set([])
    while len(score_hist) < 2 or (score_hist[-1][0] < score_hist[-2][0]) == loss:  #loss
        score = []
        
        for f in X_train.columns.tolist():  #if passing ndarray then convert to feat index
            if f not in selected_feature:        
                ohe_feat = list(selected_feature) + [f]
                
                X_ohe_train, X_ohe_val = ohe(X_train,X_val, ohe_feat)
                
                lr = lm.LinearRegression()
                lr.fit(X_ohe_train, y_train)
                val_score = np.sqrt(mean_squared_error(y_val,lr.predict(X_ohe_val)))
                
                score.append([val_score, f])
        
        score_hist.append(sorted(score, reverse=loss)[-1])
        selected_feature.add(sorted(score, reverse=loss)[-1][1])
        
    #droping last feature
    selected_feature.remove((score_hist[-1][1]))
    return selected_feature, score_hist

In [ ]:
def rf_cv_soft(X_train, y_train, X_test):
    fold =0
    skf = StratifiedKFold(n_splits=4,shuffle=True, random_state=333)
    
    y_submit=[]
    fold_score = []
    
    for train_index, test_index in skf.split(X_train, y_train):
        
        print("FOLD :{}\n".format(fold))
        
        X_model = X_train.iloc[train_index]
        y_model = y_train.iloc[train_index] 
        X_val = X_train.iloc[test_index] 
        y_val = y_train.iloc[test_index] 
        

        print("X_model : {}\t y_model : {}".format(X_model.shape, y_model.shape))
        
        #sm_enn = SMOTEENN(random_state=333)
        #
        #X_enn, y_enn = sm_enn.fit_sample(X_model, y_model)
        #X_val, y_val = sm_enn.fit_sample(X_val, y_val)
        #
        #print("\nAfter balanced dataset...")
        #print("X_model : {}\t y_model : {}\n".format(X_enn.shape, y_enn.shape))        
        
        X_enn, y_enn = X_model,y_model
        
        
        lgb_model = lgb.Dataset(data=X_enn,
                                label=y_enn,
                                feature_name=X_model.columns.tolist(),
                                free_raw_data = False
                                )
        
        lgb_val = lgb.Dataset(data=X_val,
                              label=y_val,
                              feature_name=X_model.columns.tolist(),
                              reference=lgb_model,
                              free_raw_data = False
                             )
                             
        model = lgb.train(
                        params = lgb_params.copy(),
                        train_set=lgb_model,
                        valid_sets=[lgb_model, lgb_val],
                        valid_names=['Train', 'Validation'],
                        verbose_eval=100, 
                        num_boost_round=99999,
                        early_stopping_rounds=100
                        #feval=maual_scoring,
                    )
        

        
        #model.fit(X_enn, y_enn)
        
        y_model = model.predict(X_enn, num_iteration=model.best_iteration)
        y_test = model.predict(X_val, num_iteration=model.best_iteration)
        
        #return y_model
        fold_score.append(roc_auc_score(y_val, y_test))
        if fold ==0:
             y_submit=np.array(model.predict(X_test))
        else:
             y_submit+=np.array(model.predict(X_test))
     
        print("Train roc : {}\t Test roc:{}".format(roc_auc_score(y_enn, y_model), roc_auc_score(y_val, y_test)))
        
        
        fold+=1
    return (y_submit, np.mean(fold_score))

In [ ]:
# Thanks and credited to https://www.kaggle.com/gemartin who created this wonderful mem reducer
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df